# **Installing needed packages**

## Installing libraries and fetching some scripts. **It takes $\approx$ 2 minutes.**

### $\textbf{Note:}$ The Python kernel needs to be restarted for changes to be applied. This happens automatically. If you are wondering why you are seeing a message saying "Your session crashed for an unknown reason", this is why. You can safely ignore this message!

In [ ]:
#@title Installation of packages
%%capture
!pip install -q condacolab
import condacolab
condacolab.install()
! pip install ase chemicals pubchempy
! pip install --upgrade x3dase
! sudo apt install cm-super dvipng texlive-latex-extra texlive-latex-recommended
! curl -LJO https://github.com/emartineznunez/LV_phase_diagrams/raw/main/shell_scripts/get_geom_mopac.sh
! curl -LJO https://github.com/emartineznunez/LV_phase_diagrams/raw/main/shell_scripts/get_freq_mopac.sh
! curl -LJO https://github.com/emartineznunez/LV_phase_diagrams/raw/main/shell_scripts/NM.sh
%env PATH="/content:.:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin"
%env LD_LIBRARY_PATH="/content:/usr/local/nvidia/lib:/usr/local/nvidia/lib64"
!chmod +x get_geom_mopac.sh get_freq_mopac.sh NM.sh
!conda install -c conda-forge mopac

# **Selecting the substance**

In [ ]:
#@title Retrieving the CID of the substance from PubChem https://pubchem.ncbi.nlm.nih.gov/
import pubchempy as pcp
import sys

nm = input('Identifier type: cid, name, smiles, sdf, inchi, inchikey, or formula: ')
if nm not in ['cid','name','smiles','sdf','inchi','inchikey','formula']:
  print('Please select one of the identifiers indicated below')
  sys.exit()

molA = input('Type your first molecule here: ')

queryA = pcp.get_compounds(molA,nm,record_type='3d')
cid_A = pcp.get_cids(molA)
if len(cid_A) == 0:
  print('Molecule not found')
else:
  print('CID of molecule: ',cid_A)

# **Optimization with MOPAC**

## MOPAC is a computational package that is capable of finding the mininim-energy structures of molecules using semi-empirical methods http://openmopac.net/

### Our calculations utilize PM7 semi-empirical Hamiltonian https://link.springer.com/article/10.1007/s00894-012-1667-x

### If the first optimization attempt leads to imaginary frequencies, a second attempt is made starting with a geometry $\textbf x'$ that results from a slight distortion of the original structure $\textbf x$ along the normal mode with the imaginary frequency $\textbf L_i$:

### $\textbf x'=\textbf x + \Delta x \textbf L_i$

### The final optimized structure can be visualized with X3D https://github.com/superstar54/x3dase

In [ ]:
#@title Optimization, vibrational frequencies and visualization
from x3dase.visualize import view_x3d_n
from ase.io import read
import IPython

m = queryA[0]
! echo "precise denout" > molA.mop
! echo "mol A opt" >> molA.mop
! echo "mol A opt" >> molA.mop
f = open('dumpA','w')
for i,a in enumerate(m.atoms): f.write(m.elements[i]+' '+str(a.x)+' '+str(a.y)+' '+str(a.z)+'\n')
f.close()
! cat dumpA >> molA.mop
! echo "" >> molA.mop
! echo "oldgeo oldens force thermo(298,298)" >> molA.mop
!mopac molA.mop &> molA.log

#Negative freqs??
freq1 = !./get_freq_mopac.sh molA.out | awk 'NR=1{print $1;exit}'
if (float(freq1[0])) < 0:
  ! echo "precise denout" > molA.mop
  ! echo "mol A opt" >> molA.mop
  ! echo "mol A opt" >> molA.mop
  !./NM.sh molA.out >> molA.mop
  ! echo "" >> molA.mop
  ! echo "oldgeo oldens force thermo(298,298)" >> molA.mop
  !mopac molA.mop &> molA.log
#Negative freqs??

ene_A = ! awk '/FINAL HEAT OF FORMATION/{print $6}' molA.out
zpe_A = ! awk '/ZERO POINT ENERGY/{print $4}' molA.out
sigmaA = ! awk '/SYMMETRY NUMBER/{print $NF}' molA.out

print('Energy of molecule A: %10.2f kcal/mol' % float(ene_A[0]))
print('ZPVE   of molecule A: %10.2f kcal/mol' % float(zpe_A[0]))
print('')
print('')
print('VISUALIZATION OF THE OPTIMIZED STRUCTURE')

!./get_geom_mopac.sh molA.out > molA.xyz
atoms = read('molA.xyz')
view_x3d_n(atoms,output='molA.html', bond=1.0, label=True)
IPython.display.HTML(filename='molA.html')

# **Plotting liquid-vapor phase-diagrams of ideal solutions**

### The vapor pressure of a pure substance $P^*$, whose boiling point at $P^0=1$ bar is $T^{0}$, can be obtained from the Clausius equation for any temperature $T$:

### $\ln P^*/P^0\approx -\frac{\Delta_{vap}H_m}{R}\left(\frac{1}{T}-\frac{1}{T^{0}}\right)$

The $\approx$ symbol is used because we assume that the molar volume of the gas is much greater than that of the liquid $V_{m,gas}>>V_{m,liq}$ , the gas behaves ideally, and $\Delta _{vap}H_m$ remains constant along the equilibiurm line.

In [ ]:
#@title Importing libraries and defining functions
import numpy as np
import sys
from chemicals import *
#from requests import get
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

def p(dH,T0,T):
  return p0 * np.exp(dH/R * (1/T0 - 1/T) )

def xa_l(T):
  return (ptot - p(dHB,TB0,T)) / (p(dHA,TA0,T) - p(dHB,TB0,T))

def xa_v(T):
  return xa_l(T) * p(dHA,TA0,T) / (p(dHB,TB0,T) +  (p(dHA,TA0,T) - p(dHB,TB0,T)) * xa_l(T))

def tcross(x):
  return (dHB + x * (dHA - dHB)) / (dHB / TB + x * (dHA / TA - dHB / TB))

#Chemical potentials Molar Gibbs energies G_m = Sum_i x_i * mu_i
def G(mu0,x,p,T):
  return mu0 + R * T * np.log(p * x / p0)
def Gm_l(xa,T):
  return xa * G(muA0,xa,p(dHA,TA0,T),T) + (1 - xa) * G(muB0,1-xa,p(dHB,TB0,T),T)
def Gm_g(xa,T):
  return xa * G(muA0,xa,ptot,T) + (1 - xa) * G(muB0,1-xa,ptot,T)

In [ ]:
#@title Plotting the phase diagrams
#Pressures, temperature range for the L-V phase diagram and constants
p0 = 1 # standard pressure of 1 bar
R = 8.314472e-3  # kJ/(K mol)

#Retrieve properties of molecules
moleculeA = search_chemical(molA)
TA0  = Tb(moleculeA.CASs)
TAc = Tc(moleculeA.CASs)
PcA = Pc(moleculeA.CASs)
TtA = Tt(moleculeA.CASs)
dHA = Riedel(TA0,TAc,PcA) * 1e-3

temp = 298
part = p(dHA,TA0,temp)*750.062
print('\033[1mSubstance :\033[0m ',molA)
print('\033[1mProperties:\033[0m')
print('Heat of vaporization: %10.2f kJ/mol' % dHA)
print('Boiling point:        %10.2f K' % TA0)
print('Triple  point:        %10.2f K' % TtA)
print('Vapor pressure (%3.0f K): %8.2f torr' % (temp,part))
print('')

T = np.linspace(TtA,TtA+100,100)

#Plot the P-T diagram (P vs T)
plt.rcParams['text.usetex'] = True
plt.plot(T,p(dHA,TA0,T)*750.062,'-',color='blue',label="A")
plt.ylabel(r'$P(\mathrm{torr})$',fontsize=20)
plt.xlabel(r'$T(K)$',fontsize=20)
plt.legend()
plt.title('Pressure-temperature phase-diagram for '+molA, fontsize=17)
plt.ylim(bottom=0)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.tight_layout()
plt.savefig('P_PA_PB.svg')
plt.show()

# **Plotting Gibbs energies of the different phases for different temperatures**

# Statistical therodynamics calculation of the standard-state chemical potential (Molar Gibbs energy) of each substance $\mu_m^{0}$.
### Using the Rigid-Rotor/Harmonic-Oscillator (RR/HO) model, this can be obtained from:

### $\mu_m^{0}=E_{el}+ZPVE-RT\ln \frac{q_m^{0}}{N_A}$

### where $E_{el}$ and $ZPVE$ are the electronic energy and zero-point vibrational energy (both determined in the above MOPAC calculation), and $-RT\ln (q_m^{0}/N_{A})$ is a thermal correction with:

### $\frac{q_m^{0}}{N_A} =\frac{q_{tr,m}^{0}}{N_A} \times q_{rot} \times q_{vib} = \left( \frac{2\pi mk_B T}{h^{2}} \right)^{3/2} \frac{k_BT}{P^{0}} \times 1/\sigma \left( \frac{k_BT}{hc} \right)^{3/2} \left( \frac{\pi}{\tilde{A}\tilde{B}\tilde{C}}\right)\times \prod_{i=1}^{N_v}\frac{1}{1-\exp(-hc\tilde{\nu} /k_BT)}$


In [ ]:
#@title Thermal correction in $\mu_m^{0}$


def G_m_0(temp,m,A,B,C,sigma,*freq):
  import math
  q_t = (2*math.pi*m*m_u*k_B*temp/h/h)**1.5*k_B*temp/bar2Pa
  rA  = h * c * 100 * A / k_B
  rB  = h * c * 100 * B / k_B
  rC  = h * c * 100 * C / k_B
  q_r = 1 / sigma * math.sqrt(math.pi)*(temp**1.5/math.sqrt(rA*rB*rC))
  q_v = 1

  for ele in freq:
    rv  = h * c * 100 * float(ele) / k_B
    q_v = -q_v / math.expm1(-rv/temp)
  q = q_t * q_r * q_v
  return - R * temp * np.log(q)

def pfs(temp,m,A,B,C,sigma,*freq):
  import math
  q_t = (2*math.pi*m*m_u*k_B*temp/h/h)**1.5*k_B*temp/bar2Pa
  rA  = h * c * 100 * A / k_B
  rB  = h * c * 100 * B / k_B
  rC  = h * c * 100 * C / k_B
  q_r = 1 / sigma * math.sqrt(math.pi)*(temp**1.5/math.sqrt(rA*rB*rC))
  q_v = 1

  for ele in freq:
    rv  = h * c * 100 * float(ele) / k_B
    q_v = -q_v / math.expm1(-rv/temp)
  q = q_t * q_r * q_v
  return sigma,A,B,C,q_t,q_r,q_v

In [ ]:
#@title Determining Gibbs energies

z = float( input('Choose the composition (z_A) for the determination of Gibbs energies: ') )

m_u = 1.66053886e-27      # atomic mass constant in kg
h = 6.6260693e-34         # J s
k_B = 1.3806505e-23       # J/K
c = 2.99792558e8          # speed of light in m/s
bar2Pa = 1e5          # 1 bar to N/m**2

# 1: when two phases start to co-exist at z
# 2: when the gas phase is more stable
diff = diff2 = 1e10
for ele in T:
  if abs(xa_l(ele) - z) < diff:
    diff = abs(xa_l(ele) - z)
    cot = ele
  if abs(xa_v(ele) - z) < diff2:
    diff2 = abs(xa_v(ele) - z)
    cot2 = ele

#Print the range of temperatures where the two phases co-exist
print('Temperature range (K) where L and G co-exist at this composition: %6.2f-%6.2f' % (cot,cot2))

#Number of points to plot the Molar Gibbs energies of the different phases
N = 100
# For the temperature dependence of mu0 check this:
# https://chem.libretexts.org/Bookshelves/Physical_and_Theoretical_Chemistry_Textbook_Maps/Thermodynamics_and_Chemical_Equilibrium_(Ellgen)/24%3A_Indistinguishable_Molecules_-_Statistical_Thermodynamics_of_Ideal_Gases/24.10%3A_The_Gibbs_Free_Energy_for_One_Mole_of_An_Ideal_Gas
T = np.linspace(cot,cot2,N)

# The standard-state chemical potentials of the substances in the gas phase (determine from Statistical Thermodynamics)
muA0 = np.empty(N)
muB0 = np.empty(N)

! ./get_freq_mopac.sh molA.out > dum
freqA = ! awk '{if($1<0) print -$1; else print $1}' dum
mA = ! awk '/MOLECULAR WEIGHT/{print $4;exit}' molA.out
AA = ! awk '/A =/{print $3; exit}' molA.out
BA = ! awk '/A =/{print $6; exit}' molA.out
CA = ! awk '/A =/{print $9; exit}' molA.out

! ./get_freq_mopac.sh molB.out > dum
freqB = ! awk '{if($1<0) print -$1; else print $1}' dum
mB = ! awk '/MOLECULAR WEIGHT/{print $4;exit}' molB.out
AB = ! awk '/A =/{print $3; exit}' molB.out
BB = ! awk '/A =/{print $6; exit}' molB.out
CB = ! awk '/A =/{print $9; exit}' molB.out

for Ti in T:
  index = np.where(T == Ti)[0][0]
  corrA = G_m_0(Ti,float(mA[0]),float(AA[0]),float(BA[0]),float(CA[0]),float(sigmaA[0]),*freqA)
  muA0i = (float(ene_A[0])+float(zpe_A[0]) )*4.184 + corrA
  corrB = G_m_0(Ti,float(mB[0]),float(AB[0]),float(BB[0]),float(CB[0]),float(sigmaB[0]),*freqB)
  muB0i = (float(ene_B[0])+float(zpe_B[0]) )*4.184 + corrB
  muA0[index] = muA0i
  muB0[index] = muB0i

#Plot the Gibbs energies of the different phases
plt.rcParams['text.usetex'] = True
plt.plot(T,Gm_l(z,T),'-',color='blue', label='L')
plt.plot(T,Gm_g(z,T),'-',color='red', label='G')
plt.plot(T,Gm_lg(z,T),'-',color='black', label='L+G')
plt.ylabel(r'$G_{m}(\mathrm{kJ/mol})$',fontsize=20)
plt.xlabel(r'$T(\mathrm{K})$',fontsize=20)
plt.legend()
plt.xlim(cot,cot2)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.tight_layout()
plt.savefig('Gibbs_energies.svg')
plt.show()

muA0 = np.empty(N)
muB0 = np.empty(N)
Tz=np.linspace(tcross(z)-1,tcross(z)+1,N)
for Ti in Tz:
  index = np.where(Tz == Ti)[0][0]
  corrA = G_m_0(Ti,float(mA[0]),float(AA[0]),float(BA[0]),float(CA[0]),float(sigmaA[0]),*freqA)
  muA0i = (float(ene_A[0])+float(zpe_A[0]) )*4.184 + corrA
  corrB = G_m_0(Ti,float(mB[0]),float(AB[0]),float(BB[0]),float(CB[0]),float(sigmaB[0]),*freqB)
  muB0i = (float(ene_B[0])+float(zpe_B[0]) )*4.184 + corrB
  muA0[index] = muA0i
  muB0[index] = muB0i

#Plot the Gibbs energies of the different phases
plt.rcParams['text.usetex'] = True
plt.plot(Tz,Gm_l(z,Tz),'-',color='blue', label='L')
plt.plot(Tz,Gm_g(z,Tz),'-',color='red', label='G')
plt.plot(Tz,Gm_lg(z,Tz),'-',color='black', label='L+G')
plt.ylabel(r'$G_{m}(\mathrm{kJ/mol})$',fontsize=20)
plt.xlabel(r'$T(\mathrm{K})$',fontsize=20)
plt.legend()
plt.xlim(tcross(z)-1,tcross(z)+1)
plt.xticks(np.arange(tcross(z)-0.75, tcross(z)+1.25, step=0.5),fontsize=16)
plt.yticks(fontsize=16)
plt.tight_layout()
plt.savefig('Gibbs_energies_zoom.svg')
plt.show()
